In [1]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.contracts import ContractCode, ContractInstance
from substrateinterface.exceptions import SubstrateRequestException
from dotenv import load_dotenv
import os
import requests

In [2]:
load_dotenv()

base_url = "https://rococo.api.subscan.io"
substrate_relay = SubstrateInterface(url="wss://shibuya-rpc.dwellir.com")
substrate_contract = SubstrateInterface(
    url='wss://rococo-contracts-rpc.polkadot.io')
api_key = os.getenv("API_KEY")

In [3]:
def format_balance(amount: int):
    amount = format(
        amount / 10**substrate_relay.properties.get('tokenDecimals', 0),
        ".15g")
    return f"{amount} {substrate_relay.properties.get('tokenSymbol', 'UNIT')}"


def get_account_transfers(account_address):
    url = base_url + "/api/scan/transfers"
    headers = {"Content-Type": "application/json", "X-API-Key": api_key}
    payload = {"address": account_address, "row": 100}

    response = requests.post(url, headers=headers, json=payload)
    return response.json()


def get_account_balance(account_address):
    result = substrate_relay.query("System", "Account", [account_address])
    balance = (result.value["data"]["free"] + result.value["data"]["reserved"])

    return format_balance(balance)


def get_transfer_details(extrinsic_index):
    url = base_url + "/api/scan/extrinsic"
    headers = {"Content-Type": "application/json", "X-API-Key": api_key}
    payload = {"extrinsic_index": extrinsic_index}

    response = requests.post(url, headers=headers, json=payload)
    return response.json()


def send_balance(recipient_address, amount):
    call = substrate_relay.compose_call(call_module='Balances',
                                        call_function='transfer',
                                        call_params={
                                            'dest': recipient_address,
                                            'value': amount * 10**15
                                        })

    extrinsic = substrate_relay.create_signed_extrinsic(
        call=call,
        keypair=Keypair.create_from_mnemonic(os.getenv("MNEMONIC")),
        era={'period': 64})

    try:
        receipt = substrate_relay.submit_extrinsic(extrinsic,
                                                   wait_for_inclusion=True)

        print('Extrinsic "{}" included in block "{}"'.format(
            receipt.extrinsic_hash, receipt.block_hash))

        print(receipt)

        if receipt.is_success:
            print('✅ Success, triggered events:')
            for event in receipt.triggered_events:
                print(f'* {event.value}')
        else:
            print('⚠️ Extrinsic Failed: ', receipt.error_message)

        return receipt

    except Substrate_relayRequestException as e:
        print(e)
        return False

In [4]:
os.path.join(os.getcwd(), 'assets',
                               'my_contract.json')

'c:\\Users\\asimj\\Documents\\Polkadot-Seoul\\assets\\my_contract.json'

In [5]:
menemonic = "edge lawn cost piano narrow annual movie duty forum patient check output"

In [6]:
# Upload WASM code
code = ContractCode.create_from_contract_files(
    metadata_file=os.path.join(os.getcwd(), 'my_contract/target/ink',
                               'my_contract.json'),
    wasm_file=os.path.join(os.getcwd(), 'my_contract/target/ink',
                           'my_contract.wasm'),
    substrate=substrate_relay)

# Deploy contract
print('Deploy contract...')
contract = code.deploy(keypair=Keypair.create_from_mnemonic(
    Embeddings),
                       constructor="new",
                       args={'init_value': True},
                       value=0,
                       gas_limit={'ref_time': 25990000000, 'proof_size': 1199000},
                       upload_code=True)

print(f'✅ Deployed @ {contract.contract_address}')

Deploy contract...
✅ Deployed @ aAhiHLK1dXfzCrhP7pnjVueSLRpRZfRiVNHQtz6uqxJqzqf


In [11]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.contracts import ContractCode, ContractInstance
from substrateinterface.exceptions import SubstrateRequestException
from dotenv import load_dotenv
import os
import requests

load_dotenv()

base_url = "https://shibuya.api.subscan.io"
substrate_relay = SubstrateInterface(url="wss://shibuya-rpc.dwellir.com")
api_key = os.getenv("API_KEY")
test = os.getenv("MNEMONIC")

In [14]:
contract = ContractInstance.create_from_address(
    contract_address='ZeKg7HtCRdTvtQP3dMcyjVsdoyQUPQYUzeDZA2C4nhus11W',
    metadata_file=os.path.join(os.getcwd(), 'assets',
                               'erc20.json'),
    substrate=substrate_relay)

# call total_supply
result = contract.read(Keypair.create_from_mnemonic(os.getenv("MNEMONIC")),
                       'total_supply')

print(result['result'])

{'Ok': {'flags': [], 'data': {'Ok': 10}}}
